# Npay증권에서 주가정보 수집하기_sqlalchemy로 DB에 저장하기_함수화

In [1]:
import os
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from datetime import datetime
from dbio import load_data, to_db

# dbio를 이용해서 DB에서 데이터 불러오기

In [2]:
data = load_data("stock_info_20251021", "stock_company_list20251021")

# 주가 정보 수집코드를 함수화, 1개 데이터를 바로 DB 저장

In [5]:
def naver_stock(code):
    result = {}
    cols = ["수집일", "종목명", "종목코드", "현재가", "변동금액", "변동률", "전일가", "시가", "고가", "상한가",
           "저가", "하한가", "거래량"]
    
    today = datetime.today()
    today = f"{today.year}{today.month}{today.day}"
    
    url = f"https://finance.naver.com/item/main.naver?code={code}"
    r = requests.get(url)
    soup = bs(r.content, 'lxml')
    # 종목명
    stock_name = soup.select_one("dl.blind > dd:nth-child(3)").text.split()[1]
    #현재가
    today_price = soup.select_one("dl.blind > dd:nth-child(5)").text.split()[1]
    # 변동금액
    change = soup.select_one("dl.blind > dd:nth-child(5)").text.split()[3:5]
    change = -int(change[1].replace(",", "")) if change[0] == '하락' else int(change[1].replace(",", ""))
    # 변동률
    percent = soup.select_one("dl.blind > dd:nth-child(5)").text.split()[5:]
    percent = "".join(percent)
    percent = percent.replace("플러스", "").replace("마이너스", "-").replace("퍼센트", "%")
    # 전일가
    yeaster_price = int(soup.select_one("dl.blind > dd:nth-child(6)").text.split()[1].replace(",", ""))
    # 시가
    opening_price = int(soup.select_one("dl.blind > dd:nth-child(7)").text.split()[1].replace(",", ""))
    # 고가
    hi = int(soup.select_one("dl.blind > dd:nth-child(8)").text.split()[1].replace(",", ""))
    #상한가
    top = int(soup.select_one("dl.blind > dd:nth-child(9)").text.split()[1].replace(",", ""))
    #저가
    low = int(soup.select_one("dl.blind > dd:nth-child(10)").text.split()[1].replace(",", ""))
    #하한가
    bottom = int(soup.select_one("dl.blind > dd:nth-child(11)").text.split()[1].replace(",", ""))
    #거래량
    volume = int(soup.select_one("dl.blind > dd:nth-child(12)").text.split()[1].replace(",", ""))

    values = (today, stock_name, code, today_price, change, percent, yeaster_price, opening_price, hi,
    top, low, bottom, volume)

    for key, value in zip(cols, values):
        result.setdefault(key, []).append(value)

    df = pd.DataFrame(result)
            
    to_db("stock_info_20251021", f"stock_price_{today[:4]}", df)
    
    return print(f"{stock_name} 주가정보 수집완료          ", end="\r")

        



# 데이터 수집 및 저장하기

In [6]:
for idx, code in enumerate(data['종목코드']):
    print(f"{idx+1}/{len(data['종목코드'])} 데이터 수집중", end="\r")
    try:
        int(code[:3])
        naver_stock(code)
    except Exception as e:
        print(e)
        

invalid literal for int() with base 10: 'USA'
invalid literal for int() with base 10: 'SGP'
invalid literal for int() with base 10: 'HKG'
invalid literal for int() with base 10: 'USA'
invalid literal for int() with base 10: 'HKG'
invalid literal for int() with base 10: 'JPN'
invalid literal for int() with base 10: 'USA'
invalid literal for int() with base 10: 'HKG'
invalid literal for int() with base 10: 'HKG'
invalid literal for int() with base 10: 'HKG'
invalid literal for int() with base 10: 'USA'
invalid literal for int() with base 10: 'HKG'
invalid literal for int() with base 10: 'HKG'
invalid literal for int() with base 10: 'CYM'
invalid literal for int() with base 10: 'USA'
invalid literal for int() with base 10: 'CYM'
invalid literal for int() with base 10: 'HKG'
invalid literal for int() with base 10: 'USA'
invalid literal for int() with base 10: 'CYM'
invalid literal for int() with base 10: 'CYM'
